In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Step 1: Define the directory containing PDF files
pdf_dir = r"C:\RAG\pdf"  # Path to the directory with PDFs C:\Users\bhanu\.vscode\Enfection\RAG\pdf

# Step 2: Initialize an empty list to store all documents
all_documents = []

# Step 3: Iterate over all PDF files in the directory
for file_name in os.listdir(pdf_dir):
    if file_name.endswith(".pdf"):  # Check for PDF files
        file_path = os.path.join(pdf_dir, file_name)
        
        # Load PDF content using PyPDFLoader
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        
        # Append the loaded documents to the main list
        all_documents.extend(documents)

In [7]:
poppler_path = r'C:\Release-24.08.0-0\poppler-24.08.0\Library\bin'

images = convert_from_path('pdf/1-1 - Regulatory Environment.pdf',poppler_path=poppler_path)

# Save images (optional)
for i, image in enumerate(images):
    image.save(f'page_{i}.png', 'PNG')

In [18]:
import fitz

pdf_file = 'pdf/1-1 - Regulatory Environment.pdf'
doc = fitz.open(pdf_file)

# Extract images from each page
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    pix = page.get_pixmap()
    pix.save(f'page_{page_num}.png')

AttributeError: module 'fitz' has no attribute 'open'

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500,chunk_overlap=200)
docs = text_splitter.split_documents(all_documents)

print ("total no of documments is",len(docs))

total no of documments is 163


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

from dotenv import load_dotenv
load_dotenv()


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("Hello World!")
vector[:5]

c:\RAG\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.05169878527522087,
 -0.033477481454610825,
 -0.031893402338027954,
 -0.029319265857338905,
 0.019925475120544434]

In [4]:
vectorstore = Chroma.from_documents(documents=docs,embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))


In [5]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":10})
retrieved_docs = retriever.invoke("What is a tax?")

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3,max_tokens=500)

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks"
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, say that you don't know"
    "Use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [8]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [9]:
response = rag_chain.invoke({"input":"What are the employment taxes"})
print(response["answer"])

Employment taxes vary by country and can include personal income tax, social security contributions, and other deductions like retirement annuity premiums or charitable donations through payroll.  The basis of assessment for these taxes can be the amount received in the tax year, the amount earned in the previous year, or an average of previous years' earnings.  Benefits-in-kind, such as company cars or health insurance, may also be subject to taxation.

